In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Deep Neural Networks

## Text Generation with RNN

In [2]:
###-----------------
### Import Libraries
###-----------------

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from collections.abc import Callable
from typing import Literal
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

%matplotlib inline

In [3]:
gpus = tf.config.list_physical_devices('GPU')

try:
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print (len(gpus), 'Phusical GPUs', len(logical_gpus), 'Logical GPUs')
except:
    print ('invalid device')

1 Phusical GPUs 1 Logical GPUs


In [4]:
###----------------
### Some parameters
###----------------

inpDir = '../../input'
outDir = '../output'
subDir = 'Shakespeare'
modelDir = '.../models'

RANDOM_STATE = 24 # REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results

EPOCHS = 25 # number of epochs
ALPHA = 0.01 # learning rate
TEST_SIZE = 0.2
BATCH_SIZE = 32
TRAIN_SIZE = 256 # fix size of train set sot that we have batches of same size
PATIENCE = 10

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 8),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = 'coolwarm' # plt.cm.Spectral

plt.rcParams.update(params)

## Generate Data Set
Shakespeare Dataset

In [5]:
text = open('shakespeare.txt', 'rb').read().decode(encoding='utf-8')

len(text)

1115395

In [6]:
vocab = sorted(set(text))
len(vocab)

65

In [7]:
tf.__version__

'2.14.0'

In [8]:
#vocab

In [9]:
char2idx = {u:i for i, u in enumerate (vocab)}

#char2idx

In [10]:
idx2char = np.array(vocab)

idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [11]:
text_as_int = np.array([char2idx[c] for c in text])

text_as_int.shape

(1115395,)

In [12]:
text_as_int

array([18, 47, 56, ...,  8,  0,  0])

In [13]:
text[:14]

'First Citizen:'

In [14]:
idx2char[18]

'F'

In [15]:
dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# print(list(dataset.as_numpy_iterator()))

In [16]:
seq_length = 100

example_per_epoch = len(text) // (seq_length+1)

example_per_epoch

11043

In [17]:
for i in dataset.take(10):
    print(i.numpy(), '|', idx2char[i.numpy()])

18 | F
47 | i
56 | r
57 | s
58 | t
1 |  
15 | C
47 | i
58 | t
47 | i


In [18]:
sequence = dataset.batch(seq_length + 1 ,
                         drop_remainder=True) # convert to batch

for item in sequence.take(2):

    print(item)

    print(repr(''.join(idx2char[item.numpy()] ) ) ) #take index values and convert to char

    print('\n')

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int64)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int64)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'




In [19]:
def fn_split_X_y (seq): # bring in sequence of length 101

    input_text = seq[:-1] # input is first 100 chars

    output_text = seq[1:] # output is first 100 chars

    return input_text, output_text

dataset = sequence.map(fn_split_X_y)

In [20]:
for X, y in dataset.take (2):
    print (repr(''.join(idx2char[X.numpy()] ) ) ) # X data
    print (repr(''.join(idx2char[y.numpy()] ) ) ) # y data
    print ('-'*50)

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
--------------------------------------------------
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
--------------------------------------------------


In [21]:
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

In [22]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

def build_model(vocab_size,
                embedding_dim,
                rnn_units,
                batch_size = BATCH_SIZE):

    model = tf.keras.Sequential([

        tf.keras.layers.Embedding(vocab_size,
                                  embedding_dim,
                                  batch_input_shape = [batch_size, None]),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences = True,
                            stateful = True,
                            recurrent_initializer = 'glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])

    return model

model = build_model(vocab_size,
                    embedding_dim,
                    rnn_units)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (32, None, 256)           16640     
                                                                 
 gru (GRU)                   (32, None, 1024)          3938304   
                                                                 
 dense (Dense)               (32, None, 65)            66625     
                                                                 
Total params: 4021569 (15.34 MB)
Trainable params: 4021569 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
for X, y in dataset.take(2):

    y_pred = model(X)

In [25]:
y_pred.shape

TensorShape([32, 100, 65])

In [26]:
loss_function = tf.losses.SparseCategoricalCrossentropy(from_logits= True)

In [27]:
model.compile(optimizer='adam',
              loss=loss_function)

In [28]:
chkPtPath = os.path.join(modelDir, subDir)

chkPtPrefix = os.path.join(chkPtPath, 'chkpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=chkPtPrefix, save_weights_only=True)

In [29]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

345/345 [==============================] - 15s 37ms/step - loss: 1.1838
Epoch 10/100
345/345 [==============================] - 14s 35ms/step - loss: 1.1436
Epoch 11/100
345/345 [==============================] - 14s 35ms/step - loss: 1.1020
Epoch 12/100
345/345 [==============================] - 14s 35ms/step - loss: 1.0608
Epoch 13/100
345/345 [==============================] - 14s 35ms/step - loss: 1.0231
Epoch 14/100
345/345 [==============================] - 14s 36ms/step - loss: 0.9846
Epoch 15/100
345/345 [==============================] - 14s 36ms/step - loss: 0.9505
Epoch 16/100
345/345 [==============================] - 15s 35ms/step - loss: 0.9196
Epoch 17/100
345/345 [==============================] - 15s 36ms/step - loss: 0.8942
Epoch 18/100
345/345 [==============================] - 14s 36ms/step - loss: 0.8735
Epoch 19/100
345/345 [==============================] - 14s 35ms/step - loss: 0.8527
Epoch 20/100
345/345 [==============================] - 14s 36ms/step - loss: 

KeyboardInterrupt: ignored

In [30]:
tf.train.latest_checkpoint(chkPtPath)

'.../models/Shakespeare/chkpt_46'

In [31]:
model = build_model(vocab_size,
                    embedding_dim,
                    rnn_units,
                    batch_size = 1)

model.load_weights(tf.train.latest_checkpoint(chkPtPath))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4021569 (15.34 MB)
Trainable params: 4021569 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
model.build(tf.TensorShape([1, None] ) )

In [33]:
def gen_text(model, start_string):
    num_generate = 1000

    input_eval = [char2idx[c] for c in start_string]

    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    temperature = 1,0
    model.reset_states()
    for i in range (num_generate):

        prediction = model(input_eval)

        prediction = tf.squeeze(prediction, 0)

        prediction_td = tf.random.categorical(prediction, num_samples=1)[-1,0].numpy()

        text_generated.append(idx2char[prediction_td])

    return start_string + ''.join(text_generated)


In [34]:
gen_text(model, 'ROMEO:')

'ROMEO:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\